In [71]:
import os
import shutil
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
download_dir = '/Users/ryanlee/Downloads/archive/lfw-deepfunneled/lfw-deepfunneled'
images_dir = '/Users/ryanlee/Desktop/FriendFinder.AI/images/train'

In [16]:
index = 0
for dir in os.listdir(download_dir):
    download_image_dir = os.path.join(download_dir, dir)
    if os.path.isdir(download_image_dir):
        for file in os.listdir(download_image_dir):
            downloaded_image = os.path.join(download_image_dir, file)
            image_path = os.path.join(images_dir, file)
            if os.path.isfile(downloaded_image):
                shutil.move(downloaded_image, image_path)
                print("Moved: ", file)

Moved:  German_Khan_0001.jpg
Moved:  Stefano_Gabbana_0001.jpg
Moved:  Dragan_Covic_0001.jpg
Moved:  Jeff_Hornacek_0001.jpg
Moved:  Sureyya_Ayhan_0001.jpg
Moved:  Deb_Santos_0001.jpg
Moved:  Bob_Newhart_0001.jpg
Moved:  Wang_Hailan_0001.jpg
Moved:  Paul_McNulty_0002.jpg
Moved:  Paul_McNulty_0001.jpg
Moved:  Jimmy_Iovine_0001.jpg
Moved:  Claudia_Pechstein_0001.jpg
Moved:  Claudia_Pechstein_0003.jpg
Moved:  Claudia_Pechstein_0002.jpg
Moved:  Claudia_Pechstein_0005.jpg
Moved:  Claudia_Pechstein_0004.jpg
Moved:  Ranil_Wickremasinghe_0002.jpg
Moved:  Ranil_Wickremasinghe_0003.jpg
Moved:  Ranil_Wickremasinghe_0001.jpg
Moved:  Ben_Chandler_0001.jpg
Moved:  Mark_Komara_0001.jpg
Moved:  Rand_Beers_0001.jpg
Moved:  Joanne_Woodward_0001.jpg
Moved:  John_Bond_0001.jpg
Moved:  Reginald_Hudlin_0001.jpg
Moved:  Lee_Baca_0001.jpg
Moved:  Mary-Kate_Olsen_0002.jpg
Moved:  Mary-Kate_Olsen_0003.jpg
Moved:  Mary-Kate_Olsen_0001.jpg
Moved:  Emily_Stevens_0001.jpg
Moved:  Xiang_Huaicheng_0001.jpg
Moved:  Phil

In [55]:
import pandas as pd

In [56]:
df = pd.read_csv("./CSVs/pairs.csv")

In [57]:
df.head()

,name,imagenum1,imagenum2,Unnamed: 3
0,Abel_Pacheco,1,4,NaN
1,Akhmed_Zakayev,1,3,NaN
2,Akhmed_Zakayev,2,3,NaN
3,Amber_Tamblyn,1,2,NaN
4,Anders_Fogh_Rasmussen,1,3,NaN


In [58]:
new_columns = ['pair1', 'id1', 'pair2', 'id2'] 

df.columns = new_columns
df.head()

,pair1,id1,pair2,id2
0,Abel_Pacheco,1,4,NaN
1,Akhmed_Zakayev,1,3,NaN
2,Akhmed_Zakayev,2,3,NaN
3,Amber_Tamblyn,1,2,NaN
4,Anders_Fogh_Rasmussen,1,3,NaN


In [59]:
for item in df.iloc[872]:
    print(item)

Tony_Blair
32
110
nan


In [60]:
df['pair2_numeric'] = pd.to_numeric(df['pair2'], errors='coerce')
df.dtypes

pair1             object
id1                int64
pair2             object
id2              float64
pair2_numeric    float64
dtype: object

In [61]:
df['id2'] = df.apply(lambda row: row['pair2_numeric'] if pd.notnull(row['pair2_numeric']) and pd.isnull(row['id2']) else row['id2'], axis=1)
df.head()

,pair1,id1,pair2,id2,pair2_numeric
0,Abel_Pacheco,1,4,4.0,4.0
1,Akhmed_Zakayev,1,3,3.0,3.0
2,Akhmed_Zakayev,2,3,3.0,3.0
3,Amber_Tamblyn,1,2,2.0,2.0
4,Anders_Fogh_Rasmussen,1,3,3.0,3.0


In [62]:
df['pair2'] = df.apply(lambda row: row['pair1'] if pd.notnull(row['pair2_numeric']) else row['pair2'], axis=1)
df.head()

,pair1,id1,pair2,id2,pair2_numeric
0,Abel_Pacheco,1,Abel_Pacheco,4.0,4.0
1,Akhmed_Zakayev,1,Akhmed_Zakayev,3.0,3.0
2,Akhmed_Zakayev,2,Akhmed_Zakayev,3.0,3.0
3,Amber_Tamblyn,1,Amber_Tamblyn,2.0,2.0
4,Anders_Fogh_Rasmussen,1,Anders_Fogh_Rasmussen,3.0,3.0


In [63]:
df.drop(columns=['pair2_numeric'], inplace=True)
df['id2'] = df['id2'].astype(int)
df.head()

,pair1,id1,pair2,id2
0,Abel_Pacheco,1,Abel_Pacheco,4
1,Akhmed_Zakayev,1,Akhmed_Zakayev,3
2,Akhmed_Zakayev,2,Akhmed_Zakayev,3
3,Amber_Tamblyn,1,Amber_Tamblyn,2
4,Anders_Fogh_Rasmussen,1,Anders_Fogh_Rasmussen,3


In [64]:
df['result'] = (df['pair2'] == df['pair1']).astype(int)
df.head()

,pair1,id1,pair2,id2,result
0,Abel_Pacheco,1,Abel_Pacheco,4,1
1,Akhmed_Zakayev,1,Akhmed_Zakayev,3,1
2,Akhmed_Zakayev,2,Akhmed_Zakayev,3,1
3,Amber_Tamblyn,1,Amber_Tamblyn,2,1
4,Anders_Fogh_Rasmussen,1,Anders_Fogh_Rasmussen,3,1


In [65]:
df.to_csv('dev_set.csv', index=False)

In [66]:
def format_image_name(name, id):
    if pd.notnull(id):
        if int(id) <= 9:
            return f'{images_dir}/{name}_000{int(id)}.jpg'
        elif int(id) <= 99:
            return f'{images_dir}/{name}_00{int(id)}.jpg'
        elif int(id) <= 999:
            return f'{images_dir}/{name}_0{int(id)}.jpg'
        else:
            return f'{images_dir}/{name}_{int(id)}.jpg'
    return None

In [67]:
df['pair1_path'] = df.apply(lambda row: format_image_name(row['pair1'], row['id1']), axis=1)
df['pair2_path'] = df.apply(lambda row: format_image_name(row['pair2'], row['id2']), axis=1)
df.head()

,pair1,id1,pair2,id2,result,pair1_path,pair2_path
0,Abel_Pacheco,1,Abel_Pacheco,4,1,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,/Users/ryanlee/Desktop/FriendFinder.AI/images/...
1,Akhmed_Zakayev,1,Akhmed_Zakayev,3,1,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,/Users/ryanlee/Desktop/FriendFinder.AI/images/...
2,Akhmed_Zakayev,2,Akhmed_Zakayev,3,1,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,/Users/ryanlee/Desktop/FriendFinder.AI/images/...
3,Amber_Tamblyn,1,Amber_Tamblyn,2,1,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,/Users/ryanlee/Desktop/FriendFinder.AI/images/...
4,Anders_Fogh_Rasmussen,1,Anders_Fogh_Rasmussen,3,1,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,/Users/ryanlee/Desktop/FriendFinder.AI/images/...


In [68]:
df['pair2_path'].loc[0]

'/Users/ryanlee/Desktop/FriendFinder.AI/images/train/Abel_Pacheco_0004.jpg'

In [69]:
df.drop(columns=['pair1', 'id1', 'pair2', 'id2'], inplace=True)
df = df.loc[:, ['pair1_path', 'pair2_path', 'result']]
df.head()

,pair1_path,pair2_path,result
0,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,1
1,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,1
2,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,1
3,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,1
4,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,/Users/ryanlee/Desktop/FriendFinder.AI/images/...,1


In [74]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

In [76]:
train_df.to_csv('./CSVs/train.csv')
test_df.to_csv('./CSVs/test.csv')

In [19]:
import tensorflow as tf
image = tf.io.read_file(df['pair1_path'].iloc[0])

In [20]:
image = tf.io.decode_jpeg(image)

In [21]:
new_image = tf.image.resize(image, [105, 105])
new_image = new_image / 255.0

In [77]:
import matplotlib.pyplot as plt

In [78]:
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_jpeg(image)
    image = tf.image.resize(image, [105, 105])
    image = image / 255.0

    return image

In [79]:
def create_pairs(df):
    input_a = []
    input_b = []
    label = []
    for i in range(df.shape[0]):
        image_a = preprocess_image(df['pair1_path'].iloc[i]).numpy()
        image_b = preprocess_image(df['pair2_path'].iloc[i]).numpy()
        label_value = df['result'].iloc[i]

        input_a.append(image_a)
        input_b.append(image_b)
        label.append(label_value)

    tensor_a = tf.convert_to_tensor(input_a, dtype=tf.float32)
    tensor_b = tf.convert_to_tensor(input_b, dtype=tf.float32)
    label_tensor = tf.convert_to_tensor(label, dtype=tf.float32)

    return tensor_a, tensor_b, label_tensor

In [80]:
train_a, train_b, train_labels = create_pairs(train_df)
test_a, test_b, test_labels = create_pairs(test_df)

In [81]:
train_dict = {
    'train_a': train_a,
    'train_b': train_b,
    'train_labels': train_labels
}
test_dict = {
    'test_a': test_a,
    'test_b': test_b,
    'test_labels': test_labels
}

with open('training_pairs.pkl', 'wb') as f:
    pickle.dump(train_dict, f)

with open('testing_pairs.pkl', 'wb') as f:
    pickle.dump(test_dict, f)